# 13. Preprocesado incorporando features estadísticas

In [1]:
# Configuración general para evitar errores de warnings y compatibilidad
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["RICH_NO_RICH"] = "1"
print("Configuración de entorno aplicada.")

Configuración de entorno aplicada.


In [1]:
import pyarrow.parquet as pq
import pandas as pd

# Ruta local del parquet consolidado
parquet_path = "/mnt/data/datasets/dataset_consolidado_optimizado.parquet"

# Leer solo la metadata (no carga datos)
metadata = pq.read_metadata(parquet_path)

print("✅ Archivo leído correctamente")
print(f"🧮 Número de filas: {metadata.num_rows:,}")
print(f"📊 Número de columnas: {metadata.num_columns}")
print(f"🧾 Columnas: {[metadata.schema.column(i).name for i in range(metadata.num_columns)]}")

# Abrir el archivo como ParquetFile
pf = pq.ParquetFile(parquet_path)

# Leer las primeras N filas del primer row group (sin cargar todo)
sample_table = pf.read_row_group(0)
sample_df = sample_table.to_pandas()

# Mostrar algunas filas
pd.set_option("display.max_columns", None)
print(sample_df.head(5))


✅ Archivo leído correctamente
🧮 Número de filas: 626,189,090
📊 Número de columnas: 13
🧾 Columnas: ['id', 'time', 'mag', 'mag_err', 'flux', 'flux_err', 'clase_variable', 'clase_variable_normalizada', 'mission', 'mission_id', 'source_dataset', 'label_source', 'band']
                             id           time     mag  mag_err     flux  \
0  ASASSN-V J000000.19+320847.2            HJD     mag  mag_err     flux   
1  ASASSN-V J000000.19+320847.2  2458017.73473  15.468  0.05599  2.35900   
2  ASASSN-V J000000.19+320847.2  2458018.75446   15.39  0.05292  2.53600   
3  ASASSN-V J000000.19+320847.2  2458034.87939  15.276  0.04876  2.81700   
4  ASASSN-V J000000.19+320847.2  2458035.92739  15.405  0.05350  2.50100   

   flux_err clase_variable clase_variable_normalizada mission    mission_id  \
0  flux_err             EW           Eclipsing Binary  ASASSN  ASASSN_gband   
1   0.12152             EW           Eclipsing Binary  ASASSN  ASASSN_gband   
2   0.12347             EW           Ecl

#### Ajustes en `script_1_transformer_preprocessing_optimizado_2.py`

1. **Mantener todo el flujo actual del script original**, sin modificar:

   * Agrupación por `id`
   * Limpieza y normalización de curvas
   * Aplicación de filtros de descarte
   * Generación de secuencias normalizadas y máscaras
   * Split y serialización

2. **Añadir el cálculo de nuevas features por curva**, en `process_single_curve`:

   * `median`, `iqr`, `amplitude`, `std`, `min`, `max`, `skew`, `kurtosis`, etc.
   * Se almacenarán como array adicional por curva (ej. `stats = [std, median, iqr, amplitude, min, max, skew, kurtosis]`)

3. **Modificar la clase `LightCurveDataset` para aceptar y devolver también las features estadísticas por curva**.

4. **Añadir las features estadísticas como cuarto elemento en los datasets** (`train_dataset`, `val_dataset`, `test_dataset`).

5. **Exportar los datasets como `.pt` igual que antes**, pero ahora con 4 elementos por muestra: `(secuencia, clase, máscara, features)`.

| Feature     | Descripción breve           | Comentario           |
| ----------- | --------------------------- | -------------------- |
| `std`       | Dispersión general          | ✅ útil               |
| `iqr`       | Rango intercuartílico       | ✅ robusto            |
| `amplitude` | Rango total (máx - mín)     | ✅ clave en variables |
| `median`    | Nivel base de brillo        | ✅ estable            |
| `mad`       | Desviación absoluta mediana | ✅ robusto a outliers |
| `skewness`    | Asimetría          | ✅ Útil para distinguir formas: eclipsantes vs sinusoidales. |
| `kurtosis`    | Curtosis           | ✅ Distingue si hay picos muy pronunciados o más suavidad.   |


In [ ]:
import warnings
import numpy as np
import sys
from pathlib import Path

# Añadir la raíz del proyecto al path
ROOT_DIR = Path.cwd().parent  # <- sube un nivel para alcanzar la raíz del proyecto
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

# Ignorar solo los RuntimeWarning de numpy (como overflows en reduce)
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")

from src.fase2.script_1_transformer_preprocessing_optimizado_2 import main as preprocessing_optimized_with_features

max_per_class_override={
    "Irregular": 9000,
    "Rotational": 9000,
    "Eclipsing Binary": 9000,
    "Delta Scuti": None,            # 7.550 → TODAS
    "RR Lyrae": 9000,               # 41.208 → TODAS NO
    "Young Stellar Object": None,   # 9.809 → TODAS
    "Cataclysmic": None,            # 2.080 → TODAS
    "White Dwarf": 0,               # 0 → LA ELIMINAMOS
    "Variable": 0                   # 0 → LA ELIMINAMOS
}

preprocessing_optimized_with_features(
    seq_length=25000,
    max_per_class=None, # usamos override completo
    max_per_class_override=max_per_class_override,
    parquet_batch_size=10_000_000,
    dataloader_batch_size=128,
    num_workers=16,
    #errores_csv_path=Path("../outputs/errores_mal_clasificados.csv")
)

In [3]:
import torch
import numpy as np
import pandas as pd

# Cargar dataset
dataset = torch.load("../data/train/train_dataset.pt")

# Limitar a 2000 ejemplos como máximo
num_samples = min(len(dataset), 2000)

# Extraer solo los vectores de features
features_list = []
for i in range(num_samples):
    _, _, _, features = dataset[i]
    if features.dim() == 1 and features.size(0) == 7:
        features_list.append(features)

# Convertir a numpy array
features_tensor = torch.stack(features_list)
features_np = features_tensor.numpy()

# Definir nombres
feature_names = ["std", "iqr", "amplitude", "median", "mad", "skew", "kurtosis"]

# Construir resumen
summary = pd.DataFrame({
    "feature": feature_names,
    "mean": np.mean(features_np, axis=0),
    "std": np.std(features_np, axis=0),
    "min": np.min(features_np, axis=0),
    "max": np.max(features_np, axis=0),
    "nan_count": np.isnan(features_np).sum(axis=0),
    "inf_count": np.isinf(features_np).sum(axis=0),
    "zeros_count": np.sum(features_np == 0, axis=0)
})

print(summary)


     feature          mean           std        min           max  nan_count  \
0        std  1.126377e+12  5.036000e+13  -0.609101  2.252754e+15          0   
1        iqr  1.532549e+01  4.931359e+02  -0.525688  2.115110e+04          0   
2  amplitude  5.559076e-01  4.580447e+00  -0.619319  8.089397e+01          0   
3     median -1.165967e-01  1.034070e+00  -5.776203  2.455565e+00          0   
4        mad  2.262454e+00  8.560209e+01  -0.549252  3.829006e+03          0   
5       skew  3.387466e-01  1.941506e+00 -11.842187  1.163362e+01          0   
6   kurtosis  7.411687e-01  2.001349e+00  -0.886497  7.705595e+00          0   

   inf_count  zeros_count  
0          0            0  
1          0            2  
2          0            0  
3          0            0  
4          0            8  
5          0            0  
6          0            0  


In [ ]:
import pickle
from collections import Counter
from src.utils.normalization_dict import normalize_label

# Ruta al archivo en caché
cache_path = "../data/train/grouped_data.pkl"

# Cargar el archivo en caché
with open(cache_path, "rb") as f:
    grouped_data = pickle.load(f)

# Contar las clases en el archivo
class_counts = Counter(group.iloc[0]["clase_variable_normalizada"] for group in grouped_data.values())

# Contar las clases en el archivo (normalizadas)
class_counts = Counter(normalize_label(group.iloc[0]["clase_variable_normalizada"]) for group in grouped_data.values())

# Mostrar el recuento de clases
print("\n📊 Recuento de clases en el archivo en caché (normalizadas):")
for clase, count in class_counts.items():
    print(f"{clase}: {count}")